## Extract a list of all the registered projects.

In [21]:
# extract all row items for registered companies
import asyncio
from playwright.async_api import async_playwright
import pandas as pd

async def scrape_verra_paginated():
    project_data = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()
        await page.goto("https://registry.verra.org/app/search/VCS")
        await page.wait_for_timeout(5000)

        # Select "Registered"
        await page.click('button:has-text("Status")')
        await page.wait_for_timeout(500)
        await page.click('option[title="Registered"]')
        await page.wait_for_timeout(1000)

        # Click "Search"
        await page.click('button:has-text("Search")')
        await page.wait_for_timeout(4000)

        # Loop through pages
        page_num = 1
        while True:
            print(f"🔄 Scraping page {page_num}...")
            await page.wait_for_timeout(3000)

            # Extract rows on this page
            rows = await page.query_selector_all("kendo-grid tbody > tr")

            for row in rows:
                try:
                    cols = await row.query_selector_all("td")
                    row_data = {}
                    for i, col in enumerate(cols):
                        key = f"col_{i}"
                        val = (await col.inner_text()).strip()
                        row_data[key] = val
                    project_data.append(row_data)
                except:
                    continue

            # Try to go to next page
            next_btn = await page.query_selector(f'a.k-link[aria-label="Page {page_num + 1}"]')
            if next_btn:
                await next_btn.click()
                page_num += 1
                await page.wait_for_timeout(3000)
            else:
                print("✅ No more pages.")
                break

        await browser.close()

    df = pd.DataFrame(project_data)
    df.to_csv("verra_paginated_projects.csv", index=False)
    print(f"✅ Scraped {len(df)} projects across {page_num} pages.")

# Run it
await scrape_verra_paginated()


🔄 Scraping page 1...
🔄 Scraping page 2...
🔄 Scraping page 3...
🔄 Scraping page 4...
🔄 Scraping page 5...
🔄 Scraping page 6...
🔄 Scraping page 7...
🔄 Scraping page 8...
🔄 Scraping page 9...
🔄 Scraping page 10...
🔄 Scraping page 11...
🔄 Scraping page 12...
🔄 Scraping page 13...
🔄 Scraping page 14...
🔄 Scraping page 15...
🔄 Scraping page 16...
🔄 Scraping page 17...
🔄 Scraping page 18...
🔄 Scraping page 19...
🔄 Scraping page 20...
🔄 Scraping page 21...
🔄 Scraping page 22...
🔄 Scraping page 23...
🔄 Scraping page 24...
🔄 Scraping page 25...
🔄 Scraping page 26...
🔄 Scraping page 27...
🔄 Scraping page 28...
🔄 Scraping page 29...
🔄 Scraping page 30...
✅ No more pages.
✅ Scraped 1464 projects across 30 pages.


## Scrapping all the details for regsitered projects

In [33]:
import asyncio
import pandas as pd
from playwright.async_api import async_playwright

# Step 1: Load project IDs from CSV (assumes first column is ID, no header)
df= pd.read_csv("verra_projects.csv")
project_ids = df.iloc[:, 0].astype(str).tolist()

# Final data list
scraped_data = []

async def scrape_project_detail(page, pid):
    url = f"https://registry.verra.org/app/projectDetail/VCS/{pid}"
    await page.goto(url)
    await page.wait_for_timeout(5000)

    def safe(selector):
        try:
            return selector.inner_text()
        except:
            return "N/A"

    # Try extracting each value by text label
    def get_cell_text_by_header(label):
        return page.locator(f"tr.hdr-row:has(th:has-text('{label}')) + tr.attr-row td span.text-break").first.inner_text()

    try:
        scraped_data.append({
            "Project ID": pid,
            "Proponent": await get_cell_text_by_header("Proponent"),
            "Project Status": await get_cell_text_by_header("VCS Project Status"),
            "Annual Emission Reductions": await get_cell_text_by_header("Estimated Annual Emission Reductions"),
            "Project Type": await get_cell_text_by_header("VCS Project Type"),
            "Methodology": await get_cell_text_by_header("VCS Methodology"),
            "Validator": await get_cell_text_by_header("VCS Project Validator"),
            "Registration Date": await get_cell_text_by_header("Project Registration Date"),
            "Crediting Period": await get_cell_text_by_header("Crediting Period Term")
        })
        print(f"✅ VCS{pid} scraped.")
    except Exception as e:
        print(f"⚠️ VCS{pid} failed: {e}")

async def main():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()

        for pid in project_ids:
            await scrape_project_detail(page, pid)

        await browser.close()

    # Save results
    df = pd.DataFrame(scraped_data)
    df.to_csv("verra_detailed_projects.csv", index=False)
    print(f"✅ Scraped {len(df)} projects. Saved to verra_detailed_projects.csv")

# Run the async function
await main()


✅ VCS5264 scraped.
✅ VCS5214 scraped.
✅ VCS5210 scraped.
✅ VCS5203 scraped.
✅ VCS5186 scraped.
✅ VCS5165 scraped.
✅ VCS5149 scraped.
✅ VCS5138 scraped.
✅ VCS5114 scraped.
✅ VCS5115 scraped.
✅ VCS5085 scraped.
✅ VCS5078 scraped.
✅ VCS5052 scraped.
✅ VCS5046 scraped.
✅ VCS5040 scraped.
✅ VCS5026 scraped.
✅ VCS5016 scraped.
✅ VCS5011 scraped.
✅ VCS4982 scraped.
✅ VCS4963 scraped.
✅ VCS4940 scraped.
✅ VCS4938 scraped.
✅ VCS4928 scraped.
✅ VCS4920 scraped.
✅ VCS4921 scraped.
✅ VCS4916 scraped.
✅ VCS4869 scraped.
✅ VCS4848 scraped.
✅ VCS4830 scraped.
✅ VCS4824 scraped.
✅ VCS4816 scraped.
✅ VCS4818 scraped.
✅ VCS4799 scraped.
✅ VCS4795 scraped.
✅ VCS4794 scraped.
✅ VCS4781 scraped.
✅ VCS4784 scraped.
⚠️ VCS4775 failed: Locator.inner_text: Timeout 30000ms exceeded.
Call log:
  - waiting for locator("tr.hdr-row:has(th:has-text('VCS Project Validator')) + tr.attr-row td span.text-break").first

✅ VCS4757 scraped.
✅ VCS4754 scraped.
✅ VCS4745 scraped.
✅ VCS4736 scraped.
✅ VCS4731 scraped.
✅ VCS47

### Downloading pdfs for extracted project list

In [ ]:
!pip install pandas aiohttp aiofiles playwright
!playwright install
!pip install nest_asyncio

In [1]:
import asyncio
import pandas as pd
from pathlib import Path
from playwright.async_api import async_playwright
import aiohttp
import aiofiles
import nest_asyncio

In [ ]:
async def download_file(session, url, dest):
    try:
        async with session.get(url) as resp:
            if resp.status == 200:
                async with aiofiles.open(dest, mode='wb') as f:
                    await f.write(await resp.read())
                print(f"✅ Downloaded: {dest}")
            else:
                print(f"❌ Failed to download {url}, status {resp.status}")
    except Exception as e:
        print(f"❌ Error downloading {url}: {e}")

In [ ]:
async def scrape_and_download(csv_path, output_dir):
    df = pd.read_csv(csv_path)
    project_ids = df['col_0'].dropna().astype(str).tolist()

    async with async_playwright() as p, aiohttp.ClientSession() as session:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        for project_id in project_ids:
            url = f"https://registry.verra.org/app/projectDetail/VCS/{project_id}"
            print(f"\n🌐 Accessing: {url}")
            try:
                await page.goto(url)
                await page.wait_for_selector("text=Documents", timeout=25000)
                await page.click("text=Documents")
                await page.wait_for_selector("a[href*='Project_ViewFile']", timeout=10000)
                #await page.wait_for_selector("div[role='tablist']", timeout=15000)
                #await page.click("div[role='tablist'] >> text=Documents")
                await page.wait_for_timeout(5000)

                links = await page.query_selector_all("div:below(:text('Documents')) a")
                print(f"🔎 Found {len(links)} links in Documents tab for project {project_id}")

                for link in links:
                    file_name = await link.inner_text()
                    href = await link.get_attribute("href")

                    if not file_name or not href:
                        continue

                    lower_name = file_name.lower()
                    if "pd" in lower_name:
                        folder = "pd"
                        tag = "PD"
                    elif "val" in lower_name or "verif" in lower_name:
                        folder = "verification"
                        tag = "VER"
                    elif "monitor" in lower_name or "prr" in lower_name:
                        folder = "review"
                        tag = "PRR"
                    else:
                        continue

                    if not href.startswith("http"):
                        href = "https://registry.verra.org" + href

                    dest_path = Path(output_dir) / folder / f"{tag}_{project_id}.pdf"
                    await download_file(session, href, dest_path)

            except Exception as e:
                print(f"⚠️ Error processing project {project_id}: {e}")
                continue

        await browser.close()

In [4]:
# ------- Configuration -------
csv_path = "verra_projects.csv"  # CSV must contain a 'ProjectID' column
output_dir = "verra_downloads"  # Output root folder

# Ensure subfolders exist
Path(output_dir).mkdir(exist_ok=True)
for sub in ["pd", "verification", "review"]:
    Path(output_dir, sub).mkdir(parents=True, exist_ok=True)

In [ ]:
nest_asyncio.apply()
await scrape_and_download(csv_path, output_dir)